In [ ]:
from IPython.display import Audio, display
from gtts import gTTS
import whisper

import importlib
import sys
sys.path.append("/content")

import dragontravel_chatbot_v3
importlib.reload(dragontravel_chatbot_v3)

from dragontravel_chatbot_v3 import DragonTravelBot, bookings_db

In [ ]:
model_whisper = whisper.load_model("medium")
chatbot = DragonTravelBot()

def text_to_speech(flow, text, lang="en"):
    """Convert text to speech and play it"""
    # Create a gTTS object with the text and language
    if flow == "query":
        if lang == "es":
            tts = gTTS(text=text, lang=lang, tld='com.mx')
        else:
            tts = gTTS(text=text, lang=lang, tld='us')
    else:
        if lang == "es":
            tts = gTTS(text=text, lang=lang, tld='es')
        else:
            tts = gTTS(text=text, lang=lang, tld='co.uk')
    
    # Save to a temporary file
    tts.save(f"{flow}.mp3")
    
    # Display an audio widget with the response
    return Audio(f"{flow}.mp3", autoplay=True)

# Function to interact with the chatbot via audio
def audio_chat_interaction():
    # Welcome message
    welcome = "Welcome to DragonTravel / Bienvenido a DragonTravel."
    display(welcome)

    t = 1
    
    while True:
        # Option to use text input 
        text_input = input("Type your message here / Ingrese su mensaje: ")
        
        # Detect language
        if not chatbot.language_set:
            chatbot.detect_language(text_input)
            lang = chatbot.detected_language
            chatbot.set_language(lang)
            chatbot.language_set = True

        if chatbot.current_state == "getting_feedback":
            display(text_to_speech("feedback", text_input, lang))
            user_message = model_whisper.transcribe("feedback.mp3", language=lang,)["text"]
        
        else:
            display(text_to_speech("query", text_input, lang))

            if chatbot.current_state == "collect_email" and t:
                user_message = model_whisper.transcribe("query.mp3", language=lang,)["text"]
                t = 0
            
            elif chatbot.current_state == "collect_email" and not t:
                user_message = text_input
            
            else:
                user_message = model_whisper.transcribe("query.mp3", language=lang,)["text"]
        

        print(f"\nUser message: {user_message}")
        
        response = chatbot.process_message(user_message)
        lang = chatbot.detected_language
        display(text_to_speech("response", response, lang))
        print(f"Response message: {response}")

        if text_input.lower() == 'exit' or text_input.lower() == 'salir':
            break

Loading NLP models...
Bot initialized and ready!


In [16]:
audio_chat_interaction()

'Welcome to DragonTravel / Bienvenido a DragonTravel.'


User message:  I need the price for 3 round trip tickets from Los Angeles to Tokyo, July 5th to July 20th.
Welcome to DragonTravel! Where would you like to fly from?


Response message: I understand you're looking for a flight with Departure: Los Angeles, Arrival: Tokyo, Trip type: round_trip.
I couldn't understand that date. Please specify a departure date (e.g., October 15, 2025).



User message:  July 5, 2026


Response message: Departing on July 05, 2025. Is this a one-way or round-trip flight?



User message:  Exit.


Response message: Is this a one-way or round-trip flight?


In [10]:
bookings_db

[{'num_passengers': 2,
  'flight_type': 'round_trip',
  'departure_airport': 'SEA',
  'arrival_airport': 'BCN',
  'departure_datetime': datetime.datetime(2025, 12, 10, 0, 0),
  'arrival_datetime': datetime.datetime(2025, 1, 12, 0, 0),
  'airline': 'DragonAir',
  'seat_class': 'Business',
  'email': 'memayl@hotmail.com',
  'booking_id': 1001,
  'created_at': datetime.datetime(2025, 3, 14, 19, 45, 41, 157611)},
 {'booking_id': 1001,
  'audio_transcription': ' I hate your English accent!',
  'language': 'en',
  'category': 'Queja',
  'timestamp': datetime.datetime(2025, 3, 14, 19, 46, 17, 608428)}]